In [35]:
import requests
import pandas as pd
from datetime import datetime
from zeep import Client, helpers

### Loading in the parameters 

In [36]:
API = pd.read_csv("api.csv")['api'].values[0]
PRODUCTION_URL = "https://openweb.nlb.gov.sg/OWS/CatalogueService.svc?singleWsdl"
client = Client(wsdl=PRODUCTION_URL)

### Functions 

In [44]:
def search_parameters(search_dict: dict) -> list:
    output_list = list()
    for key, value in search_dict.items():
        output_list.append(
            { "SearchField": key, "SearchTerms": value }
        )
    return output_list

In [45]:
search_parameters({'Title':'Robust Python'})

[{'SearchField': 'Title', 'SearchTerms': 'Robust Python'}]

#### Search Feature 

In [4]:
search_input = {
    "APIKey": API,
    "SearchItems": {
        "SearchItem": [
            {
                "SearchField": "Title", 
                "SearchTerms": "Robust Python : write clean and maintainable code"
            },
        ]
    },
    "Modifiers": {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 1,
        "SetId": None
    }
}

In [5]:
client.service.Search(**search_input)

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'TotalRecords': 1,
    'NextRecordPosition': 0,
    'SetId': '364863875',
    'Titles': {
        'Title': [
            {
                'BID': '205499632',
                'ISBN': '9781098100667 (paperback)|1098100662 (paperback)',
                'TitleName': 'Robust Python : write clean and maintainable code / Patrick Viafore.',
                'Author': 'Viafore, Patrick,',
                'PublishYear': '2021',
                'MediaCode': 'BK',
                'MediaDesc': 'Books'
            }
        ]
    }
}

#### Available Titles by Library 

In [37]:
# bid_no = 9781101595954
isbn = 9781101595954
get_avail = {
    "APIKey": API,
    "ISBN": isbn,
    # "BID": bid_no,
    "Modifiers" : {
        "SortSchema": None,
        "StartRecordPosition": 1,
        "MaximumRecords": 100,
        "SetId": None
    },
}

In [38]:
avail_info = client.service.GetAvailabilityInfo(**get_avail)

In [39]:
avail_info

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'NextRecordPosition': 0,
    'SetId': '365515007',
    'Items': {
        'Item': [
            {
                'ItemNo': None,
                'BranchID': 'DIGITAL',
                'BranchName': 'Digital Library',
                'LocationCode': None,
                'LocationDesc': None,
                'CallNumber': None,
                'StatusCode': 'S',
                'StatusDesc': 'Not on Loan',
                'MediaCode': None,
                'MediaDesc': None,
                'StatusDate': '22/05/2021',
                'DueDate': None,
                'ClusterName': None,
                'CategoryName': None,
                'CollectionCode': 'OVERDRIVE',
                'CollectionMinAgeLimit': None
            }
        ]
    }
}

In [40]:
df = pd.DataFrame()
for i in pd.DataFrame(helpers.serialize_object(avail_info).get("Items").values()).T[0]:
    df = df.append(pd.DataFrame.from_dict(i, orient='index').T)
df['bid'] = bid_no
df.head()

,ItemNo,BranchID,BranchName,LocationCode,LocationDesc,CallNumber,StatusCode,StatusDesc,MediaCode,MediaDesc,StatusDate,DueDate,ClusterName,CategoryName,CollectionCode,CollectionMinAgeLimit,bid
0,None,DIGITAL,Digital Library,None,None,None,S,Not on Loan,None,None,22/05/2021,None,None,None,OVERDRIVE,None,9781101595954


#### Get Title Details 

In [117]:
title_inputs = {
    "APIKey": API,
    "BID": 204485571,
}

In [118]:
title_details = client.service.GetTitleDetails(**title_inputs)
title_details

{
    'Status': 'OK',
    'Message': 'Operation completed successfully',
    'ErrorMessage': None,
    'TitleDetail': {
        'BID': '204485571',
        'TitleName': '40 algorithms every programmer should know : hone your problem-solving skills by learning different algorithms and their implementation in Python / Imran Ahmad.',
        'Author': 'Ahmad, Imran,',
        'OtherAuthors': None,
        'Publisher': None,
        'PhysicalDesc': 'x, 365 pages :illustrations ;24 cm',
        'Subjects': {
            'Subject': [
                'Computer programs',
                'Python (Computer program language)'
            ]
        },
        'Summary': None,
        'Notes': '\r\nIncludes index.',
        'ISBN': '1789801214 (paperback)|9781789801217 (paperback)',
        'ISSN': None,
        'NTitleName': None,
        'NAuthor': None,
        'NPublisher': None
    }
}

#### Read Recommendations - Doesn't seem to work

In [168]:
# get_recom = {
#     "APIKey": API,
#     "BID": 204485571,
#     "Modifiers" : {
#         "SortSchema": None,
#         "StartRecordPosition": 1,
#         "MaximumRecords": 100,
#         "SetId": None
#     },
# }

# PRODUCTION_URL = "http://openweb.nlb.gov.sg/ows/ReadAlikeService.svc"
# client = Client(wsdl=PRODUCTION_URL)
# vars(client.wsdl)

# get_recommendations = client.readalikeservice.GetRecommendationsForTitles(**get_recom)
# get_recommendations